### settings.py

In [3]:
# Customizable settings

''' RECURRING VARIABLES '''

import numpy as np
import nibabel as nib
import ciftools_FA as ct
import pandas as pd

# directory containing subdirectories named fter subject IDs that contain the timeseries and surface files
root_dir = "/home/fralberti/Data/HCP_S1200"
# directory where all intermediate files and the final output will be saved
output_dir = "/home/fralberti/Documents/BlackBox/Prj_Network-variability/Results"
# path to cognitive data
cog_path = f'{root_dir}/unrestricted_fralberti_4_8_2022_8_22_23.csv'
# .txt containing IDs of subjects to include in the analysis
f = open(f'{root_dir}/Subjects/subj_IDs_338.txt', 'r')
subj_id = np.array(f.read().splitlines(), dtype='int32')
# group name
group = '338'

# Schaefer atlas to use for analyses
lbl_N = 400
nw_N = 7
networks_txt = f'/home/fralberti/Data/Shaefer2018_HCP/Schaefer2018_{lbl_N}Parcels_{nw_N}Networks_order_info.txt'
networks = nib.load(f'/home/fralberti/Data/Shaefer2018_HCP/Schaefer2018_{lbl_N}Parcels_{nw_N}Networks_order.dlabel.nii')
NW_tbl = ct.agg_networks(networks, networks, func=np.median, by_hemisphere=False, label_tbl=True)[1]
nw_name = NW_tbl.groupby('name').agg(np.median).sort_values('network').index.to_list()[1:]

# number of jobs for parallelized operations
nj = -5


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
/tmp/ipykernel_39077/4260995017.py:28: FutureWarning: The operation <function median at 0x7f56c00a43a0> failed on a column. If any error is raised, this will raise an exception in a future version of pandas. Drop these columns to avoid this warning.
  nw_name = NW_tbl.groupby('name').agg(np.median).sort_values('network').index.to_list()[1:]


### cifti_to_npy.py

In [5]:
# Import dependencies

import os
import nibabel as nib
import numpy as np
from scipy import stats
import pandas as pd
import ciftools_FA as ct
import statsmodels.api as sm
from joblib import Parallel, delayed
# from settings import root_dir, output_dir, subj_id, group, lbl_N, nw_N, networks, NW_tbl, nw_name, nj
subj_dir = f"{root_dir}/Subjects/"
gcca_dir = f"{output_dir}/GCCA"
#----------------------------------------------------------------------------------------------------

''' Save vertex/parcel/network-level GCCA (individual and group median) from dscalar.nii to .npy arrays'''

# Write function to run in parallel
def fun(subj, root_dir, output_dir, nw_N, lbl_N):
    grad = nib.load(f'{root_dir}/{subj}/Analysis/{subj}.GCCA_525.32k_fs_LR.dscalar.nii')
    NW_df, NW_tbl = ct.agg_networks(grad, networks, func=np.median, by_hemisphere=False, label_tbl=True)
    NW_df.iloc[0, :] = 0
    NW_tbl.iloc[0, :] = 0
    lbl_df = ct.agg_labels(grad, networks, func=np.median)    
    
    # Append subject's vertex, label, and network-level gradients to group list (for SD)
    grad_NW = NW_df.T.values
    grad_lbl = lbl_df.T.values
    grad_vtx = np.asarray(grad.get_fdata())
    
    # Save median individual median gradient of parcels and networks
    np.save(f'{output_dir}/{subj}.gcca.{nw_N}NWs', NW_df) 
    np.save(f'{output_dir}/{subj}.gcca.{lbl_N}Parc', lbl_df)
    
    return grad_NW, grad_lbl, grad_vtx
    
#--------------------------------------------------------------------------------------------------

# Create directory

if not os.path.exists(gcca_dir):
	os.mkdir(gcca_dir, mode = 0o777)

r = Parallel(n_jobs=nj)(delayed(fun)(subj, subj_dir, gcca_dir, nw_N, lbl_N) for subj in subj_id)
# unpack output
grad_NW, grad_lbl, grad_vtx = zip(*r)

# Save parcels' and networks' median gradient of all subjects
np.save(f'{gcca_dir}/{group}.gcca.{nw_N}NWs', np.array(grad_NW)) 
np.save(f'{gcca_dir}/{group}.gcca.{lbl_N}Parc', np.array(grad_lbl))
np.save(f'{gcca_dir}/{group}.gcca.32k_fs_LR', np.array(grad_vtx)) 



pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67. 

KeyboardInterrupt: 

### interindividual_dispersion.py

In [ ]:
# Import dependencies

import os
import nibabel as nib
import numpy as np
from scipy import stats
import pandas as pd
import ciftools_FA as ct
import statsmodels.api as sm
from joblib import Parallel, delayed
# from settings import root_dir, output_dir, subj_id, group, lbl_N, nw_N, networks, NW_tbl, nw_name, nj
subj_dir = f"{root_dir}/Subjects/"
gcca_dir = f"{output_dir}/GCCA"
#----------------------------------------------------------------------------------------------------



''' Calculate dispersion of connectivity in 3D gradient space'''

gcca_nw = np.load(f'{gcca_dir}/{group}.gcca.{nw_N}NWs.npy')
gcca_parc = np.load(f'{gcca_dir}/{group}.gcca.{lbl_N}Parc.npy')
gcca_vtx = np.load(f'{gcca_dir}/{group}.gcca.32k_fs_LR.npy')

# AVVERAGE SQUARED DISTANCE from group centroid
def gcca_dispersion(gcca):
    centroids = gcca.mean(axis=0)
    squares = np.square(gcca - centroids)
    sum_squares = squares[:, :2, :].sum(axis=1)
    distance = np.sqrt(sum_squares)
    avg_distance2 = np.square(distance).sum(axis=0) / distance.shape[0]
    avg_components2 = squares.sum(axis=0) / squares.shape[0]
    return np.vstack([avg_distance2, avg_components2])

dispersion_nw = gcca_dispersion(gcca_nw)
dispersion_parc = gcca_dispersion(gcca_parc)
dispersion_vtx = gcca_dispersion(gcca_vtx)

np.save(f'{output_dir}/{group}.gcca_dispersion.{nw_N}NWs', dispersion_nw)
np.save(f'{output_dir}/{group}.gcca_dispersion.{lbl_N}Parc', dispersion_parc)
np.save(f'{output_dir}/{group}.gcca_dispersion.32k_fs_LR', dispersion_vtx)
#----------------------------------------------------------------------------------------------------



''' Save dscalar files'''

# Load gcca data
gcca_nw = np.load(f'{gcca_dir}/{group}.gcca.{nw_N}NWs.npy')
gcca_parc = np.load(f'{gcca_dir}/{group}.gcca.{lbl_N}Parc.npy')
gcca_vtx = np.load(f'{gcca_dir}/{group}.gcca.32k_fs_LR.npy')

# Get vertex network labels
_, lbl_tbl = ct.agg_networks(networks, networks, by_hemisphere=False, label_tbl=True)
vtx_nw = lbl_tbl.set_index('label').loc[networks.get_fdata().squeeze().astype('int32'), 'network'].values
template = nib.load(f'{subj_dir}/{subj_id[0]}/Analysis/{subj_id[0]}.GCCA_525.32k_fs_LR.dscalar.nii')

# Vertex - gradient median
scalars = np.median(gcca_vtx, axis=0)
out = f'{output_dir}/{group}.gcca.32k_fs_LR.dscalar.nii'
ct.save_dscalar(scalars, template, out, names=['Gradient_1', 'Gradient_2', 'Gradient_3'])

# Parcel - gradient median
scalars = np.median(gcca_parc[:, :, networks.get_fdata().squeeze().astype('int32')], axis=0)
out = f'{output_dir}/{group}.gcca.{lbl_N}Parc.32k_fs_LR.dscalar.nii'
ct.save_dscalar(scalars, networks, out, names=['Gradient 1', 'Gradient 2', 'Gradient 3'])

# Network - gradient median
scalars = np.median(gcca_nw[:, :, vtx_nw], axis=0)
out = f'{output_dir}/{group}.gcca.{lbl_N}Parc_{nw_N}NWs.32k_fs_LR.dscalar.nii'
ct.save_dscalar(scalars, networks, out, names=['Gradient_1', 'Gradient_2', 'Gradient_3'])


dispersion_nw = np.load(f'{output_dir}/{group}.gcca_dispersion.{nw_N}NWs.npy')
dispersion_parc = np.load(f'{output_dir}/{group}.gcca_dispersion.{lbl_N}Parc.npy')
dispersion_vtx = np.load(f'{output_dir}/{group}.gcca_dispersion.32k_fs_LR.npy')

# Vertex - dispersion 
scalars = dispersion_vtx
out = f'{output_dir}/{group}.gcca_dispersion.32k_fs_LR.dscalar.nii'
ct.save_dscalar(scalars, template, out, names=['All', 'Gradient_1', 'Gradient_2', 'Gradient_3'])

# Parcel - dispersion 
scalars = dispersion_parc[:, networks.get_fdata().squeeze().astype('int32')]
out = f'{output_dir}/{group}.gcca_dispersion.{lbl_N}Parc.32k_fs_LR.dscalar.nii'
ct.save_dscalar(scalars, networks, out, names=['All', 'Gradient_1', 'Gradient_2', 'Gradient_3'])

# Network - dispersion 
scalars = dispersion_nw[:, vtx_nw]
out = f'{output_dir}/{group}.gcca_dispersion.{lbl_N}Parc_{nw_N}NWs.32k_fs_LR.dscalar.nii'
ct.save_dscalar(scalars, networks, out, names=['All', 'Gradient_1', 'Gradient_2', 'Gradient_3'])



### variability_clusyters.py

In [ ]:
# Import dependencies

import os
import nibabel as nib
import numpy as np
from scipy import stats
import pandas as pd
import ciftools_FA as ct
import statsmodels.api as sm
import matplotlib.pyplot as plt
import subprocess as sp
from joblib import Parallel, delayed
from settings import root_dir, output_dir, subj_id, group, lbl_N, nw_N, networks, networks_txt, NW_tbl, nw_name, nj
subj_dir = f"{root_dir}/Subjects/"
gcca_dir = f"{output_dir}/GCCA"
#----------------------------------------------------------------------------------------------------



''' Generate clusters of highest inter-individual variability '''

# Find variance clusters
dscalar = f"{output_dir}/{group}.gcca_dispersion.32k_fs_LR.dscalar.nii"
percentile = 95
clusters = []
for g_tmp in range(4):
    thr = np.percentile(np.load(f"{output_dir}/{group}.gcca_dispersion.32k_fs_LR.npy"), percentile, axis=1)[g_tmp]
    srf_min = 200
    vol_min = 1
    out = f"{output_dir}/{group}.gcca_disp_clusters.32k_fs_LR.dscalar.nii"
    L_srf = f"{root_dir}/HCP_S1200_GroupAvg_v1/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii"
    R_srf = f"{root_dir}/HCP_S1200_GroupAvg_v1/S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii"

    sp.run(f"wb_command -cifti-find-clusters {dscalar} {thr} {srf_min} {thr} {vol_min} COLUMN {out} -left-surface {L_srf} -right-surface {R_srf}", shell=True)

    clusters_g = nib.load(f"{output_dir}/{group}.gcca_disp_clusters.32k_fs_LR.dscalar.nii").get_fdata()[g_tmp, :]
    clusters.append(clusters_g)

clusters = np.asarray(clusters)
clusters = stats.rankdata(clusters, axis=1, method='dense') - 1

ct.save_dscalar(clusters, nib.load(dscalar), out, names=['All', 'Gradient_1', 'Gradient_2', 'Gradient_3'])
#----------------------------------------------------------------------------------------------------



''' Save clusters as dlabel.nii file '''

clusters = nib.load(f"{output_dir}/{group}.gcca_disp_clusters.32k_fs_LR.dscalar.nii")


labels_path = f"{output_dir}/{group}.gcca_disp_clusters.32k_fs_LR.dscalar.nii"
labels_array = nib.load(labels_path)
lbl_txt =  f'{output_dir}/{group}.gcca_disp_clusters_info.txt'
new_dlabel = f'{output_dir}/{group}.gcca_disp_clusters_info.dlabel.txt'


labels = np.unique(clusters.get_fdata()).astype('int32')
new_labels = []
colors = np.round(plt.cm.Dark2(range(len(labels)-1))* 255).astype('int32').astype('str')
for lbl in labels[labels!=0]:
    new_labels.append(f"Disp_cluster_{lbl}")
    new_labels.append(' '.join(np.hstack([lbl, colors[lbl-1,:]])))

#with open(lbl_txt, 'w') as f:
#    #lines = f.readlines()
#    lines.extend([s + '\n' for s in new_labels])
#    f.close()

with open(lbl_txt, 'w') as f:
    f.write('\n'.join(new_labels))
    
    
# Create dlabel file
new_dlabel = f'{output_dir}/{group}.gcca_disp_clusters.32k_fs_LR.dlabel.nii'
cmd = f'wb_command -cifti-label-import {labels_path} {lbl_txt} {new_dlabel} -discard-others'
sp.run(cmd, shell=True)

#----------------------------------------------------------------------------------------------------



''' Add dispersion cluster to Schaefer atlas '''

clusters = nib.load(f"{output_dir}/{group}.gcca_disp_clusters.32k_fs_LR.dscalar.nii")

# allign network and cluster maps
shared_l = np.isin(ct.struct_info('CIFTI_STRUCTURE_CORTEX_LEFT', networks)[2], ct.struct_info('CIFTI_STRUCTURE_CORTEX_LEFT', clusters)[2])
shared_r = np.isin(ct.struct_info('CIFTI_STRUCTURE_CORTEX_RIGHT', networks)[2], ct.struct_info('CIFTI_STRUCTURE_CORTEX_RIGHT', clusters)[2])
shared_vv = np.hstack([shared_l, shared_r])

clusters = clusters.get_fdata()[0].astype('int32')
clusters[clusters != 0] += 1000    ### add 1000 to differentiate from original parcels
cluster_N = len(np.unique(clusters))
parcels = networks.get_fdata()[0, shared_vv].astype('int32')
parcels[clusters!=0] = clusters[clusters!=0]
removed = np.unique(networks.get_fdata()[0, shared_vv].astype('int32'))[~np.isin(np.unique(networks.get_fdata()[0, shared_vv].astype('int32')), parcels)]

# save dscalar with new parcellation
out = f'{output_dir}/{group}.dispROIs_Schaefer2018_{lbl_N}Parcels_{nw_N}Networks.dscalar.nii'
template = nib.load(f"{output_dir}/{group}.gcca_disp_clusters.32k_fs_LR.dscalar.nii")
ct.save_dscalar(parcels, template, out)


# Create list of new parcels and colors
new_labels = []
names = [f"{nw_N}Networks_LH_ROI_{i}" for i in range(1,cluster_N)]
colors = np.round(plt.cm.Dark2(range(np.unique(clusters[clusters!=0]).size))* 255).astype('int32').astype('str')
for i, lbl in enumerate(np.unique(clusters[clusters!=0])):
    new_labels.append(names[i]+'\n')
    new_labels.append(' '.join(np.hstack([lbl, colors[i,:], '\n'])))
   
    
# Save new label-list-file with added parcels
orig_lbl_txt = networks_txt
with open(orig_lbl_txt, 'r+') as f:
    lines = f.readlines()
    lines.extend(new_labels)
    f.close()

for i in range(1, len(lines), 2):
    if i > len(lines):
        break
    label = int(lines[i].split()[0])
    if label not in parcels:
        del lines[i], lines[i-1]

new_lbl_txt = f'{output_dir}/{group}.dispROIs_Schaefer2018_{lbl_N}Parcels_{nw_N}Networks.txt'
with open(new_lbl_txt, 'w') as f:
    f.truncate(0)
    f.write(''.join(lines))
    f.close()
    
# Create dlabel file
new_dlabel = f'{output_dir}/{group}.dispROIs_Schaefer2018_{lbl_N}Parcels_{nw_N}Networks.dlabel.nii'
cmd = f'wb_command -cifti-label-import {out} {new_lbl_txt} {new_dlabel}'
sp.run(cmd, shell=True)

#----------------------------------------------------------------------------------------------------



''' Generate previous tables and images using the integrated parcellation '''

joint_atlas = nib.load(f'{output_dir}/{group}.dispROIs_Schaefer2018_{lbl_N}Parcels_{nw_N}Networks.dlabel.nii')
labels = np.unique(joint_atlas.get_fdata()).astype('int32')

# Parcel - gradient median
gcca_parc = []
for subj in subj_id:
    grad = nib.load(f'{subj_dir}/{subj}/Analysis/{subj}.GCCA_525.32k_fs_LR.dscalar.nii')
    lbl_df = ct.agg_labels(grad, joint_atlas, func=np.median)
    gcca_parc.append(lbl_df.T.values)
    np.save(f'{output_dir}/{subj}.gcca.dispROIs_{lbl_N}Parc', lbl_df)
    
gcca_parc = np.array(gcca_parc)
np.save(f'{output_dir}/{group}.gcca.dispROIs_{lbl_N}Parc', gcca_parc)

parc_idx = stats.rankdata(joint_atlas.get_fdata().squeeze(), method='dense') - 1
scalars = np.median(gcca_parc[:, :, parc_idx], axis=0)
out = f'{output_dir}/{group}.gcca.dispROIs_{lbl_N}Parc.32k_fs_LR.dscalar.nii'
ct.save_dscalar(scalars, joint_atlas, out, names=['Gradient 1', 'Gradient 2', 'Gradient 3'])


# Parcel - dispersion 
dispersion_vtx = nib.load(f'{output_dir}/{group}.gcca_dispersion.32k_fs_LR.dscalar.nii')
dispersion_parc = ct.agg_labels(dispersion_vtx, joint_atlas, func=np.median)
np.save(f'{output_dir}/{group}.gcca_dispersion.dispROIs_{lbl_N}Parc', dispersion_parc)

scalars = joint_atlas.get_fdata().copy()

for lbl, val in dispersion_parc.iterrows():
    scalars[:,scalars[0]==lbl] = val.iloc[0]
    
out = f'{output_dir}/{group}.gcca_dispersion.dispROIs_{lbl_N}Parc.32k_fs_LR.dscalar.nii'
ct.save_dscalar(scalars, joint_atlas, out, names=['All'])



pixdim[1,2,3] should be non-zero; setting 0 dims to 1
/home/fralberti/Code/interindividual-gradient-dispersion/gradient-variability/settings.py:28: FutureWarning: The operation <function median at 0x7f8e003a0a60> failed on a column. If any error is raised, this will raise an exception in a future version of pandas. Drop these columns to avoid this warning.
  nw_name = NW_tbl.groupby('name').agg(np.median).sort_values('network').index.to_list()[1:]


[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67.  47. ... 339. 339. 339.]]
[[196.  67. 

### generate_csv

In [6]:
# Import dependencies

import os
import nibabel as nib
import numpy as np
from scipy import stats
import pandas as pd
import ciftools_FA as ct
import statsmodels.api as sm
import matplotlib.pyplot as plt
import subprocess as sp
from joblib import Parallel, delayed
#from settings import root_dir, output_dir, subj_id, cog_path, group, lbl_N, nw_N, networks, networks_txt, NW_tbl, nw_name, nj
subj_dir = f"{root_dir}/Subjects/"
gcca_dir = f"{output_dir}/GCCA"
#----------------------------------------------------------------------------------------------------



''' Generate table with vertex affiliations, gradients, and dispersion measures '''

grad_SD = nib.load(f'{output_dir}/{group}.gcca_dispersion.32k_fs_LR.dscalar.nii')
cifti_59k = nib.load(f'{subj_dir}/{subj_id[0]}/Analysis/{subj_id[0]}.GCCA_525.32k_fs_LR.dscalar.nii')

# Create parcel label array
parc_lbl = []
vv = []
hemi = []
for struct in ['CIFTI_STRUCTURE_CORTEX_LEFT', 'CIFTI_STRUCTURE_CORTEX_RIGHT']:
    NWos, NWn, NWvv = ct.struct_info(struct, networks)
    Gos, Gn, Gvv = ct.struct_info(struct, cifti_59k)
    shared_vv = np.isin(NWvv, Gvv)
    parc_lbl.extend(networks.get_fdata()[0, NWos : NWos + NWn][shared_vv])
    vv.extend(NWvv[shared_vv])
    h = (0 if struct=='CIFTI_STRUCTURE_CORTEX_LEFT' else 1)
    hemi.extend(np.repeat(h, Gn))


# Create network label array
parc_to_NW = ct.agg_networks(networks, networks, func='mean', by_hemisphere=False, label_tbl=True)[1]
NW_lbl = parc_to_NW.set_index('label').loc[parc_lbl, 'network'].values

# Create dataframe for tests
disp_df = pd.DataFrame(np.vstack([vv, hemi, parc_lbl, NW_lbl, grad_SD.get_fdata()]).T, columns = ['Vtx', 'Hemi', 'Parc', 'NW', 'Disp_tot', 'Disp_G1', 'Disp_G2', 'Disp_G3'])

# Add ROIs labels
clusters = nib.load(f"{output_dir}/{group}.gcca_disp_clusters.32k_fs_LR.dscalar.nii").get_fdata().T
disp_df[['DispROI_tot', 'DispROI_G1', 'DispROI_G2', 'DispROI_G3']] = clusters


disp_df.to_csv(f'{output_dir}/{group}.gcca_dispersion.csv', index=False)

#----------------------------------------------------------------------------------------------------



''' Generate table with subject data for analyses '''

# Load cognitive scores
cog_df = pd.read_csv(cog_path,
                    usecols = ['Subject', 'PicVocab_Unadj', 'ReadEng_Unadj', 'CardSort_Unadj', 'Flanker_Unadj', 'ProcSpeed_Unadj',
                               'VSPLOT_TC', 'PMAT24_A_CR', 'PicSeq_Unadj', 'ListSort_Unadj', 'IWRD_TOT',
                               'CogEarlyComp_Unadj', 'CogFluidComp_Unadj', 'CogCrystalComp_Unadj', 'CogTotalComp_Unadj',
                               'ER40_CR'],
                     index_col = 'Subject').loc[subj_id,:]

# Calculate weighted factors
cog_df['G'] = (cog_df[['PicVocab_Unadj', 'ReadEng_Unadj', 'CardSort_Unadj', 'Flanker_Unadj', 'ProcSpeed_Unadj',
                       'VSPLOT_TC', 'PMAT24_A_CR', 'PicSeq_Unadj', 'ListSort_Unadj', 'IWRD_TOT']]
               * [.624, .642, .364, .259, .232, .578, .626, .354, .451, .294]).mean(axis=1)


# Load covariates for correcting cognitive scores
covar_df = pd.merge(pd.read_csv(cog_path,
                                usecols=['Subject', 'Gender'],
                                dtype={'Subject':'int32', 'Gender':'category'}),
                    pd.read_csv(f'{root_dir}/RESTRICTED_arianna_9_7_2022_8_13_11.csv',
                                usecols=['Subject', 'Age_in_Yrs', 'Handedness', 'SSAGA_Educ'],
                                dtype={'Subject':'int32', 'Age_in_Yrs':'int32', 'Handedness':'int32'}),
                    ).set_index('Subject')

covar_df = covar_df.loc[subj_id]
cog_df = cog_df.merge(covar_df, left_index=True, right_index=True)


# Add aggregate gradient value of the ROIs
ROIs = pd.read_csv(f'{output_dir}/{group}.gcca_dispersion.csv')
cog_df.index = cog_df.index.astype('str')

grads = np.load(f'{gcca_dir}/{group}.gcca.32k_fs_LR.npy').copy()

for grd in ['tot', 'G1', 'G2', 'G3']:
    for i in range(3):
        grads_df = pd.DataFrame(np.vstack([ROIs[f"DispROI_{grd}"], grads[:, i, :]]).T, columns=np.hstack(["ROI", subj_id]))
        grads_avg = grads_df.groupby("ROI").agg(np.median).reset_index().T.iloc[:,1:].drop('ROI')
        cols = [f"G{i+1}_ROI{int(n)}_Disp{grd}" for n in grads_avg.columns]
        grads_avg = grads_avg.rename(columns=dict(zip(grads_avg.columns, cols))).rename_axis('Subject')
        cog_df = cog_df.merge(grads_avg, left_index=True, right_index=True)



cog_df.to_csv(f'{output_dir}/{group}.cog_data.csv')

#----------------------------------------------------------------------------------------------------



''' Measure clusters' overlap with networks '''

disp_df = pd.read_csv(f"{output_dir}/{group}.gcca_dispersion.csv", header=0, usecols=["Vtx", "NW", f"DispROI_tot"])
disp_df = disp_df[disp_df[f"DispROI_tot"] > 0]

clusters = disp_df.groupby([f"DispROI_tot", "NW"]).agg("count")["Vtx"].reset_index().pivot(index=f"DispROI_tot", columns="NW", values="Vtx")
clusters.columns = np.array(nw_name)[clusters.columns.astype("int32")-1]
clusters.loc["TOT", :] = clusters.sum()
clusters.loc[:, "TOT"] = clusters.sum(axis=1)

clusters.to_csv(f"{output_dir}/{group}.dispROIs_{nw_N}NWs_overlap.csv")


missing_NWs = np.asarray(nw_name)[~np.isin(nw_name, clusters.columns)]
missing_NWs = pd.DataFrame(np.full([len(clusters),  len(missing_NWs)], np.nan), columns=missing_NWs, index=clusters.index)
clusters = clusters.merge(missing_NWs, left_index=True, right_index=True)[np.hstack([nw_name, 'TOT'])]

clusters = (clusters.T / clusters.T.sum() * 100).T * 2
titles = ['L TPJ', 'L vlPFC', 'L OP', 'L dlPFC', 'R TPJ', 'R vlPFC', 'R dlPFC', 'R OP']

clusters.rename(columns={'SalVentAttn':'VAN', 'DorsAttn':'DAN', 'Default':'DMN', 'Cont':'FPN'}, index=dict(zip(clusters.index, titles)), inplace=True)
clusters = clusters.loc[['L TPJ', 'L vlPFC', 'L dlPFC', 'L OP', 'R TPJ', 'R vlPFC', 'R dlPFC', 'R OP'], :'TOT']
clusters[clusters.isna()] = 0

print(clusters)


               Vis  SomMot        DAN        VAN  Limbic        FPN  \
DispROI_tot                                                           
L TPJ          0.0     0.0  18.489583  41.145833     0.0   0.000000   
L vlPFC        0.0     0.0  18.343195  57.396450     0.0  21.301775   
L dlPFC        0.0     0.0   0.000000  15.822785     0.0   8.860759   
L OP         100.0     0.0   0.000000   0.000000     0.0   0.000000   
R TPJ          0.0     0.0  29.881657  34.615385     0.0   0.000000   
R vlPFC        0.0     0.0   1.081081  14.234234     0.0  58.738739   
R dlPFC        0.0     0.0   0.000000   3.171247     0.0  50.528541   
R OP         100.0     0.0   0.000000   0.000000     0.0   0.000000   

                   DMN    TOT  
DispROI_tot                    
L TPJ        40.364583  100.0  
L vlPFC       2.958580  100.0  
L dlPFC      75.316456  100.0  
L OP          0.000000  100.0  
R TPJ        35.502959  100.0  
R vlPFC      25.945946  100.0  
R dlPFC      46.300211  100.0  
R

### modeling_intelligence.py

In [7]:
# Import dependencies

import os
import nibabel as nib
import numpy as np
from scipy import stats
import pandas as pd
import ciftools_FA as ct
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import subprocess as sp
from joblib import Parallel, delayed
#from settings import root_dir, output_dir, subj_id, group, lbl_N, nw_N, networks, networks_txt, NW_tbl, nw_name, nj
subj_dir = f"{root_dir}/Subjects/"
gcca_dir = f"{output_dir}/GCCA"
#----------------------------------------------------------------------------------------------------



''' Define functions to parallelize permutation testing '''

def perm_fit(model, data, perm_cols):
    data.loc[:, perm_cols] = np.random.permutation(data.loc[:, perm_cols])
    lm_perm = ols(model, data=data).fit()
    return lm_perm.fvalue, lm_perm.tvalues

def perm_lm(model, data, perm_n, perm_cols=None, n_jobs=1):
    lm = ols(model, data=data).fit()

    A = np.identity(len(lm.params))
    A = A[1:,:]
    F_results = {'Endog': lm.model.endog_names, 'F': lm.fvalue, 'R2': lm.rsquared, 'R2_adj': lm.rsquared_adj}
    dfs = [lm.params, lm.tvalues, lm.bse]
    t_results = pd.DataFrame(np.vstack(dfs).T, index=lm.model.exog_names, columns=['Coef', 't', 'SE'])
    
    F_null = []
    t_null = []
    data_perm = data.copy()
    if perm_cols is None:
        perm_cols = data.columns[data.columns.isin(lm.model.exog_names)]
        
    r= Parallel(n_jobs=n_jobs)(delayed(perm_fit)(model, data_perm, perm_cols) for i in range(perm_n))
    F_null, t_null = zip(*r)
    
    t_p = [sum(np.abs(np.asarray(t_null).T[n]) > np.abs(t_results.t[n])) / perm_n for n in range(len(t_results))]
    t_results['p'] = t_p
    F_results['p'] = (sum(F_null > F_results['F']) / perm_n)

    return F_results, t_results, np.array(F_null), np.array(t_null)
#----------------------------------------------------------------------------------------------------



''' Run permutation tests '''

# Aggregate highly correlated predictors
cog_df = pd.read_csv(f'{output_dir}/{group}.cog_data.csv', index_col=0, header=0)
G = 'G1'
cog_df[f'{G}_ROI38_Disptot'] = np.mean(cog_df[[f'{G}_ROI3_Disptot', f'{G}_ROI7_Disptot']], axis=1)
cog_df[f'{G}_ROI15_Disptot'] = np.mean(cog_df[[f'{G}_ROI1_Disptot', f'{G}_ROI5_Disptot']], axis=1)
G = 'G2'
cog_df[f'{G}_ROI38_Disptot'] = np.mean(cog_df[[f'{G}_ROI3_Disptot', f'{G}_ROI7_Disptot']], axis=1)
cog_df[f'{G}_ROI15_Disptot'] = np.mean(cog_df[[f'{G}_ROI1_Disptot', f'{G}_ROI5_Disptot']], axis=1)
G = 'G3'
cog_df[f'{G}_ROI38_Disptot'] = np.mean(cog_df[[f'{G}_ROI3_Disptot', f'{G}_ROI7_Disptot']], axis=1)
cog_df[f'{G}_ROI15_Disptot'] = np.mean(cog_df[[f'{G}_ROI1_Disptot', f'{G}_ROI5_Disptot']], axis=1)


comp_cols = ['CogFluidComp_Unadj', 'CogCrystalComp_Unadj', 'G']
ROI_cols = np.array(['G1_ROI2_Disptot', 'G1_ROI4_Disptot', 'G1_ROI6_Disptot', 'G1_ROI7_Disptot', 'G1_ROI15_Disptot', 'G1_ROI38_Disptot'])
covars = ['C(Gender)', 'Age_in_Yrs', 'Handedness', 'SSAGA_Educ']


# Correct for covariates
X = ' + '.join(covars)
for col in comp_cols:
    lm = ols(f'{col} ~ {X}', data=cog_df).fit()
    cog_df[col] = stats.zscore(lm.resid, axis=0)


# Run permutation tests
perm_n = 10000
X = ' + '.join(ROI_cols)
F_tests = []
F_test_p = []

results = {}
for y in comp_cols:
    F, t, F_null, t_null = perm_lm(f'{y} ~ {X}', cog_df, perm_n, perm_cols=None, n_jobs=-2)
    results[y] = {'F': F, 't': t, 'F_null': F_null, 't_null': t_null}
    
    
# Correct multiple comparisons
pF_adj = sm.stats.multipletests([values['F']['p'] for key, values in results.items()], method='fdr_bh', alpha=0.05)[1]

for i, key in enumerate(results.keys()):
    results[key]['F']['p_adj'] = pF_adj[i]
    results[key]['t']['p_adj'] = np.hstack(['',sm.stats.multipletests(results[key]['t']['p'][1:], method='fdr_bh', alpha=0.05)[1]])

global_results = pd.DataFrame([results['CogFluidComp_Unadj']['F'], results['CogCrystalComp_Unadj']['F'], results['G']['F']])

print(global_results)

for key, value in results.items():
    if results[key]['F']['p_adj'] < 0.05:
        print(f"post-hoc:\t{key}\n", pd.DataFrame(results[key]['t']), '\n')
        
#----------------------------------------------------------------------------------------------------
        

''' Test correlation with individual tests '''        
        
# test correlation between Fluid intelligence and its components
r, p = stats.spearmanr(cog_df[['CardSort_Unadj', 'Flanker_Unadj', 'PMAT24_A_CR', 'PicSeq_Unadj', 'ListSort_Unadj', 'CogFluidComp_Unadj']], nan_policy='omit')

cols = ['CardSort_Unadj', 'Flanker_Unadj', 'PMAT24_A_CR', 'PicSeq_Unadj', 'ListSort_Unadj', 'CogFluidComp_Unadj', 'G1_ROI6_Disptot']
r, p = stats.spearmanr(cog_df[cols], nan_policy='omit')
r = pd.DataFrame(r, index=cols, columns=cols)
p = pd.DataFrame(p, index=cols, columns=cols)

h, pAdj, _, _ = sm.stats.multipletests(p.loc['G1_ROI6_Disptot', 'CardSort_Unadj':'ListSort_Unadj'], method='fdr_bh')

results = pd.DataFrame(np.vstack([r.loc['G1_ROI6_Disptot', 'CardSort_Unadj':'ListSort_Unadj'], pAdj, h]).T,
             columns=['rho', 'p_adj', 'h'], index=['CardSort_Unadj', 'Flanker_Unadj', 'PMAT24_A_CR', 'PicSeq_Unadj', 'ListSort_Unadj'])

print(results)



                  Endog         F        R2    R2_adj       p   p_adj
0    CogFluidComp_Unadj  2.751444  0.047643  0.030327  0.0128  0.0384
1  CogCrystalComp_Unadj  0.956891  0.017050 -0.000768  0.4624  0.4624
2                     G  2.132093  0.037210  0.019758  0.0504  0.0756
post-hoc:	CogFluidComp_Unadj
                       Coef         t        SE       p                 p_adj
Intercept         0.429412  1.725382  0.248879  0.0772                      
G1_ROI2_Disptot  -0.918436 -0.682881  1.344943  0.4827               0.70524
G1_ROI4_Disptot   0.914885  0.559184  1.636106  0.5877               0.70524
G1_ROI6_Disptot  -5.174402 -2.649952  1.952640  0.0079  0.047400000000000005
G1_ROI7_Disptot  -0.627346 -0.255340  2.456906  0.7939                0.7939
G1_ROI15_Disptot -1.284491 -1.037619  1.237922  0.3001                0.6002
G1_ROI38_Disptot -2.597633 -1.246108  2.084597  0.2121                0.6002 

                     rho     p_adj    h
CardSort_Unadj -0.137122  0.0290

### graph_analysis.py

In [7]:
# Import dependencies

import os
import nibabel as nib
import numpy as np
from scipy import stats
import pandas as pd
import ciftools_FA as ct
import statsmodels.api as sm
import matplotlib.pyplot as plt
import subprocess as sp
from joblib import Parallel, delayed
import networkx as nx
from settings import root_dir, output_dir, subj_id, group, lbl_N, nw_N, networks, networks_txt, NW_tbl, nw_name, nj
subj_dir = f"{root_dir}/Subjects/"
gcca_dir = f"{output_dir}/GCCA"
#----------------------------------------------------------------------------------------------------


''' Generate FC graphs and extract topological metrics '''

thr = 90

NWs_n_ROIs = nib.load(f'{output_dir}/{group}.dispROIs_Schaefer2018_{lbl_N}Parcels_{nw_N}Networks.dlabel.nii')

modules_df = ct.agg_networks(NWs_n_ROIs, NWs_n_ROIs)[1][1:].reset_index()

communities = [set(modules_df.label.astype('int32')[modules_df.network==nw].values) for nw in modules_df.network.unique()]
nodes = dict( zip( range( len(modules_df.index) ), modules_df.label) )

def graph_diagnostics(subj, root_dir, communities, thr):
    M = np.genfromtxt(f'{subj_dir}/{subj}/Analysis/{subj}.REST_All_fcMatrix.csv', delimiter=',')[1:, 1:]
    G = nx.from_numpy_matrix(M)
    G.threshold(thr)
    G = nx.relabel_nodes(G, nodes)
    G = nx.algorithms.full_diagnostics(G, modules=communities, swi=False)
    nx.write_gpickle(G,f'{subj_dir}/{subj}/Analysis/{subj}.rfMRI_graph_{thr}.Schaefer_400parcs.gpickle')
    return [subj]

_ = Parallel(n_jobs=nj, prefer='processes')(delayed(graph_diagnostics)(subj, root_dir, communities, thr) for subj in subj_id)

#----------------------------------------------------------------------------------------------------


''' Save graph metrics of interest to cifti '''

attributes = ['strength', 'clustering', 'global_e', 'local_e', 'between_c', 'participation_c']

def get_graph(subj):
    return nx.read_gpickle(f'{subj_dir}/{subj}/Analysis/{subj}.rfMRI_graph_{thr}.Schaefer_400parcs.gpickle')

individual_maps = []
for subj in subj_id:
    G = get_graph(subj)
    metrics = np.array([list(nx.get_node_attributes(G, attribute).values()) for attribute in attributes])
    
    isolates = nx.isolates(G)
    metrics[:,np.isin(G, list(isolates))] = np.nan
    individual_maps.append(metrics)
    
    
individual_maps = np.array(individual_maps)
np.save(f'{output_dir}/{group}.graph_metrics.dispROIs_{lbl_N}Parc', individual_maps)

graph_dispersion_maps = stats.iqr(individual_maps, 0, rng=(25, 75), nan_policy='omit') 
graph_dispersion_df = pd.DataFrame(graph_dispersion_maps.T, columns=attributes)
graph_dispersion_df.to_csv(f'{output_dir}/{group}.graph_metrics_dispersion.IQR_Schaefer2018_{lbl_N}Parc_{nw_N}NW.csv', index=False)


joint_atlas = nib.load(f'{output_dir}/{group}.dispROIs_Schaefer2018_{lbl_N}Parcels_{nw_N}Networks.dlabel.nii')
labels = joint_atlas.get_fdata().copy().squeeze().astype('int32')
label_idx = stats.rankdata(labels, method='dense').squeeze() -1 


scalars = np.zeros([graph_dispersion_df.shape[1], labels.size]).T

for lbl, val in graph_dispersion_df.iterrows():
    scalars[label_idx==lbl+1, :] = val.values

out = f'{output_dir}/{group}.graph_dispersion.dispROIs_{lbl_N}Parc.32k_fs_LR.dscalar.nii'
ct.save_dscalar(scalars.T, joint_atlas, out, names=attributes)



pixdim[1,2,3] should be non-zero; setting 0 dims to 1
/home/fralberti/Code/interindividual-gradient-dispersion/gradient-variability/settings.py:28: FutureWarning: The operation <function median at 0x7fe430132a60> failed on a column. If any error is raised, this will raise an exception in a future version of pandas. Drop these columns to avoid this warning.
  nw_name = NW_tbl.groupby('name').agg(np.median).sort_values('network').index.to_list()[1:]


### metric_maps.py

In [8]:
# Import dependencies

import os
import nibabel as nib
import numpy as np
from scipy import stats
import pandas as pd
import ciftools_FA as ct
import statsmodels.api as sm
import matplotlib.pyplot as plt
import subprocess as sp
from joblib import Parallel, delayed
import networkx as nx
from settings import root_dir, output_dir, subj_id, group, lbl_N, nw_N, networks, networks_txt, NW_tbl, nw_name, nj
subj_dir = f"{root_dir}/Subjects/"
gcca_dir = f"{output_dir}/GCCA"
#----------------------------------------------------------------------------------------------------


''' Test correlation between dispersion of graph metrics and principal gradient '''

gcca_dispersion = np.load(f'{output_dir}/{group}.gcca_dispersion.dispROIs_{lbl_N}Parc.npy')[:,1:]
gcca_median = np.median(np.load(f'{output_dir}/{group}.gcca.dispROIs_{lbl_N}Parc.npy')[:,0,1:].squeeze(), axis=0)
graph_dispersion_df = pd.read_csv(f'{output_dir}/{group}.graph_metrics_dispersion.IQR_Schaefer2018_{lbl_N}Parc_{nw_N}NW.csv')

graph_dispersion_df['gcca_disp'] = gcca_dispersion[1:,0]
graph_dispersion_df['gcca'] = gcca_median
graph_dispersion_df[graph_dispersion_df==0] = np.nan


results = {}

for i, graph_dispersion in enumerate(graph_dispersion_df.T.iterrows()):
    if i == 6:
        break
    x = graph_dispersion_df['gcca_disp']
    y = graph_dispersion[1]
    spearman_r = stats.spearmanr(x, y, nan_policy='omit')
    
    results[graph_dispersion[0]] = {'r': spearman_r[0],
                                    'p': spearman_r[1]}

p_unadj = [value['p'] for key, value in results.items()]
p_adj = sm.stats.multipletests(p_unadj, method='fdr_bh')[1]

for i, key in enumerate(results):
    results[key]['p_adj'] = p_adj[i]
    
print(pd.DataFrame(results).T)

#----------------------------------------------------------------------------------------------------


''' Test cross-subject correlation between graph metrics and principal gradient '''

attributes = ['strength', 'clustering', 'global_e', 'local_e', 'between_c', 'participation_c']


gcca = np.load(f'{output_dir}/{group}.gcca.dispROIs_{lbl_N}Parc.npy')[:,:,1:]
gmetrics = np.load(f'{output_dir}/{group}.graph_metrics.dispROIs_{lbl_N}Parc.npy')

results = {attribute:{'r':[], 'p':[]} for attribute in attributes}

for i, parc_metrics in enumerate(gmetrics[:, :, :].T):
    for j, metric in enumerate(parc_metrics):
        r, p = stats.spearmanr(metric, gcca[:,0,i], nan_policy='omit')
        results[attributes[j]]['r'].extend(np.array([r]))
        results[attributes[j]]['p'].extend(np.array([p]))
        
for metric, dictionary in results.items():
    h, p, _, _ = sm.stats.multipletests(dictionary['p'], alpha=0.05, method='fdr_by')
    dictionary['p'] = p
    dictionary['r'] = dictionary['r']
    
scalars = [results[metric][index] for metric in results.keys() for index in results[metric].keys()]
scalars = np.array(scalars)


joint_atlas = nib.load(f'{output_dir}/{group}.dispROIs_Schaefer2018_{lbl_N}Parcels_{nw_N}Networks.dlabel.nii')
labels = joint_atlas.get_fdata().copy().squeeze().astype('int32')
label_idx = stats.rankdata(labels, method='dense').squeeze() -1 

maps_array = np.zeros([scalars.shape[0], labels.size])

for i, scalar in enumerate(scalars):
    for lbl in np.unique(label_idx):
        if lbl==0:
            continue
        maps_array[i, label_idx==lbl] = scalar[lbl-1]
    
out = f'{output_dir}/{group}.graph_metric_correlation.dispROIs_{lbl_N}Parc.32k_fs_LR.dscalar.nii'
names = np.asanyarray([[f'{metric}_R', f'{metric}_pval'] for metric in results]).reshape(-1).tolist()
ct.save_dscalar(maps_array, joint_atlas, out, names=names)


                        r             p         p_adj
strength        -0.112497  2.339245e-02  2.807094e-02
clustering      -0.474630  2.041424e-23  1.224854e-22
global_e         0.031853  5.221680e-01  5.221680e-01
local_e         -0.353881  5.236228e-13  1.570868e-12
between_c        0.173850  4.859970e-04  7.289954e-04
participation_c -0.250144  4.914576e-07  9.829153e-07


In [ ]:
gcca = np.load(f'{output_dir}/{group}.gcca.dispROIs_{lbl_N}Parc.npy')[:,:,1:]
gmetrics = np.load(f'{output_dir}/{group}.graph_metrics.dispROIs_{lbl_N}Parc.npy')